# <span style = 'color: red'>**Práctica uno.** Aprendizaje de máquina. *Redes neuronales*</span>

# ***Parte uno.*** **CNN y LeNET5.**

## **Simón Cuartas Rendón**

En este *notebook* de Python se aborda el desarrollo del **primer problema** de la *práctica dos* del curso de **Aprendizaje de Máquina**.

A continuación, se abren los paquetes que se usarán para dar solución al problema propuesto.

**Anotación.** Este *notebook* está disponible en línea para facilitar su revisión. Se puede acceder al repositorio de GitHub [aquí](https://github.com/scuartasr/aprendmaq_entregable_2/tree/main), y en particular, a este cuaderno aquí.